In [1]:
import pandas as pd
import numpy as np
import os
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
inicio = time.time()

data_files = [x for x in os.listdir('./data')]

all_matches = pd.DataFrame()
all_teams = pd.DataFrame()
all_rounds = pd.DataFrame()
all_KDA = pd.DataFrame()
all_kills = pd.DataFrame()
all_economy = pd.DataFrame()
all_RPS = pd.DataFrame()
all_locations = pd.DataFrame()
all_events = pd.DataFrame()

for x in data_files:
    df_matches = pd.read_excel('./data/'+ x, sheet_name='Matches')
    all_matches = pd.concat([all_matches, df_matches])
    df_teams = pd.read_excel('./data/'+ x, sheet_name='Teams')
    all_teams = pd.concat([all_teams, df_teams])
    df_rounds = pd.read_excel('./data/'+ x, sheet_name='Rounds')
    all_rounds = pd.concat([all_rounds, df_rounds])
    df_KDA = pd.read_excel('./data/'+ x, sheet_name='KDA')
    all_KDA = pd.concat([all_KDA, df_KDA])
    df_kills = pd.read_excel('./data/'+ x, sheet_name='Kills')
    all_kills = pd.concat([all_kills, df_kills])
    df_economy = pd.read_excel('./data/'+ x, sheet_name='Economy')
    all_economy = pd.concat([all_economy, df_economy])
    df_RPS = pd.read_excel('./data/'+ x, sheet_name='Round Player Stats')
    all_RPS = pd.concat([all_RPS, df_RPS])
    df_locations = pd.read_excel('./data/'+ x, sheet_name='Locations')
    all_locations = pd.concat([all_locations, df_locations])
    df_events = pd.read_excel('./data/'+ x, sheet_name='Events')
    all_events = pd.concat([all_events, df_events])
    
final = time.time()
tiempo_total = final - inicio
print('Tiempo total de ejecución: ', tiempo_total)

Tiempo total de ejecución:  223.5461413860321


In [3]:
matches = pd.read_excel('matches.xlsx')
matches.head()

,eventId,seriesId,matchId,map,teamId,teamName,oppTeamId,oppTeamName,atkFirstTeamId,atkFirst,teamScore,won
0,566,14278,29398,Ascent,1137,VIRT,3975,KCP,1137,1,3,0
1,566,14278,29398,Ascent,3975,KCP,1137,VIRT,1137,0,13,1
2,566,14278,29399,Bind,1137,VIRT,3975,KCP,3975,0,5,0
3,566,14278,29399,Bind,3975,KCP,1137,VIRT,3975,1,13,1
4,566,14277,29395,Split,141,C9,281,RISE,141,1,7,0


Iremos recopilando información desde los distintos dataframes generados anteriormente, de la tabla RPS (Round Player Stats) seleccionamos las siguiente columnas.

In [4]:
all_RPS.columns

Index(['Unnamed: 0', 'matchId', 'roundId', 'roundNumber', 'playerId', 'teamId',
       'side', 'acs', 'kills', 'firstKills', 'deaths', 'firstDeaths',
       'assists', 'damage', 'headshots', 'bodyshots', 'legshots', 'plants',
       'defusals', 'clutches', 'clutchOpponents', 'clutchOpportunities',
       'impact', 'kastRounds', 'seriesId'],
      dtype='object')

In [5]:
all_RPS.shape

(143520, 25)

In [6]:
df_2 = all_RPS[['matchId','teamId', 'acs', 'kills', 'firstKills', 'deaths', 'firstDeaths',
                'assists', 'damage', 'headshots', 'bodyshots', 'legshots', 'plants',
                'defusals', 'clutches', 'clutchOpponents', 'clutchOpportunities']]

In [7]:
df_2.head()

,matchId,teamId,acs,kills,firstKills,deaths,firstDeaths,assists,damage,headshots,bodyshots,legshots,plants,defusals,clutches,clutchOpponents,clutchOpportunities
0,29398,3975,489,2,0,1,0,0,215,2,5,1,0,0,0,0,0
1,29398,3975,55,0,0,0,0,1,55,0,1,0,0,0,0,0,0
2,29398,1137,0,0,0,1,0,0,0,0,0,0,0,0,0,4,1
3,29398,3975,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,29398,3975,139,0,0,0,0,2,139,0,0,0,0,0,0,0,0


Se agrupa por la ID de la partida y por la ID del equipo, para las variables 'acs' y 'damage' se obtiene la media y para las demás la suma.

In [8]:
df_2 = df_2.groupby(['matchId','teamId']).agg({'acs':'mean', 'kills':'sum','firstKills':'sum', 
                                               'deaths':'sum', 'firstDeaths':'sum','assists':'sum',
                                               'damage':'mean', 'headshots':'sum','bodyshots':'sum',
                                               'legshots':'sum', 'plants':'sum','defusals':'sum',
                                               'clutches':'sum', 'clutchOpponents':'sum','clutchOpportunities':'sum'})

A partir de las columnas seleccionadas, se crean nuevas calculando 'KDR', es la relación entre kills y deaths, 'KD' es la diferencia entre kills y deaths, 'FKD' es la diferencia entre primera kill y deaths, y 'KDA' es la suma de kills y assists sobre deaths.

In [9]:
df_2['KDR'] = df_2['kills']/df_2['deaths']
df_2['KD'] = df_2['kills']-df_2['deaths']
df_2['FKD'] = df_2['firstKills']-df_2['firstDeaths']
df_2['KDA'] = df_2['kills']+df_2['assists']/df_2['deaths']

In [10]:
df_2.head()

acs  kills  firstKills  deaths  firstDeaths  assists  \
matchId teamId                                                                
28309   590     211.663636     80           9      73           13       44   
        2990    189.445455     73          13      80            9       25   
28310   590     218.400000     84          11      70           10       29   
        2990    198.704762     70          10      84           11       33   
28922   279     212.560000     78          13      55            7       31   

                    damage  headshots  bodyshots  legshots  plants  defusals  \
matchId teamId                                                                 
28309   590     133.572727         58        212        29      10         1   
        2990    120.327273         51        183        15       6         0   
28310   590     130.485714         54        187        20       5         4   
        2990    120.952381         37        187        16       9         0   
28922   279     131.260000         59        149         8      11         1   

                clutches  clutchOpponents  clutchOpportunities       KDR  KD  \
matchId teamId                                                                 
28309   590            0               31                    9  1.095890   7   
        2990           2               43                   14  0.912500  -7   
28310   590            3               33                   11  1.200000  14   
        2990           2               45                   15  0.833333 -14   
28922   279            1               24                    8  1.418182  23   

                FKD        KDA  
matchId teamId                  
28309   590      -4  80.602740  
        2990      4  73.312500  
28310   590       1  84.414286  
        2990     -1  70.392857  
28922   279       6  78.563636

Luego, de la tabla de datos económicos seleccionamos los indicadores que podamos agrupar para poder añadir al dataset.

In [11]:
all_economy.columns

Index(['Unnamed: 0', 'roundId', 'roundNumber', 'playerId', 'agent', 'score',
       'weapon', 'armorId', 'remainingCreds', 'spentCreds', 'loadoutValue',
       'matchId', 'teamId', 'weaponCategory', 'minCredsNextRound'],
      dtype='object')

In [12]:
df_3 = all_economy[['matchId','teamId','spentCreds','loadoutValue']]

In [13]:
df_3 = df_3.groupby(['matchId','teamId']).agg({'spentCreds':'mean', 'loadoutValue':'mean'})

In [14]:
df_3.head()

spentCreds  loadoutValue
matchId teamId                           
28309   590     2353.181818   3571.818182
        2990    2480.454545   3557.272727
28310   590     2673.809524   3752.380952
        2990    2561.428571   3508.095238
28922   279     2422.000000   3725.500000

De la base de matches, seleccionamos las columnas que darán inicio al dataset final. Se identifica que no todos los equipos tienen nobre asignado, por lo que estos valores vacío se completan con 'UNNAMED'.

In [15]:
matches = matches[['eventId','seriesId','matchId','teamId','teamName','map','oppTeamId',
                   'oppTeamName','teamScore','atkFirst','won']]

In [16]:
matches['teamName'].fillna('UNNAMED', inplace=True)
matches['oppTeamName'].fillna('UNNAMED', inplace=True)

In [17]:
matches.head(10)

,eventId,seriesId,matchId,teamId,teamName,map,oppTeamId,oppTeamName,teamScore,atkFirst,won
0,566,14278,29398,1137,VIRT,Ascent,3975,KCP,3,1,0
1,566,14278,29398,3975,KCP,Ascent,1137,VIRT,13,0,1
2,566,14278,29399,1137,VIRT,Bind,3975,KCP,5,0,0
3,566,14278,29399,3975,KCP,Bind,1137,VIRT,13,1,1
4,566,14277,29395,141,C9,Split,281,RISE,7,1,0
5,566,14277,29395,281,RISE,Split,141,C9,13,0,1
6,566,14277,29396,141,C9,Ascent,281,RISE,11,0,0
7,566,14277,29396,281,UNNAMED,Ascent,141,C9,13,1,1
8,566,14276,29392,95,NYFU,Breeze,318,GENG,5,0,0
9,566,14276,29392,318,GENG,Breeze,95,NYFU,13,1,1


In [18]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428 entries, 0 to 1427
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   eventId      1428 non-null   int64 
 1   seriesId     1428 non-null   int64 
 2   matchId      1428 non-null   int64 
 3   teamId       1428 non-null   int64 
 4   teamName     1428 non-null   object
 5   map          1428 non-null   object
 6   oppTeamId    1428 non-null   int64 
 7   oppTeamName  1428 non-null   object
 8   teamScore    1428 non-null   int64 
 9   atkFirst     1428 non-null   int64 
 10  won          1428 non-null   int64 
dtypes: int64(8), object(3)
memory usage: 122.8+ KB


Unimos las 2 tablas agrupadas anteriormente con la base de matches.

In [19]:
df_valorant = pd.merge(matches, df_2, on=['matchId','teamId'])

In [20]:
df_valorant = pd.merge(df_valorant, df_3, on=['matchId','teamId'])

Creamos una nueva columna con el indicador de puntuación de economía, el cual corresponde al daño realizado por cada 1000 créditos gastados.

In [21]:
df_valorant['avgEconRating'] = (df_valorant['damage']/df_valorant['spentCreds'])*1000

In [22]:
df_valorant.head()

,eventId,seriesId,matchId,teamId,teamName,map,oppTeamId,oppTeamName,teamScore,atkFirst,...,clutches,clutchOpponents,clutchOpportunities,KDR,KD,FKD,KDA,spentCreds,loadoutValue,avgEconRating
0,566,14278,29398,1137,VIRT,Ascent,3975,KCP,3,1,...,0,47,13,0.594203,-28,-4,41.202899,2536.875000,2989.375000,35.245134
1,566,14278,29398,3975,KCP,Ascent,1137,VIRT,13,0,...,2,12,5,1.682927,28,4,69.658537,2355.000000,3820.625000,61.820594
2,566,14278,29399,1137,VIRT,Bind,3975,KCP,5,0,...,2,54,15,0.537500,-37,-10,43.150000,2452.777778,2871.666667,37.879955
3,566,14278,29399,3975,KCP,Bind,1137,VIRT,13,1,...,0,14,5,1.860465,37,10,80.418605,2179.444444,3838.333333,61.366301
4,566,14277,29395,141,C9,Split,281,RISE,7,1,...,1,45,14,0.752941,-21,-4,64.341176,2618.500000,3167.500000,43.681497


Se crea diccionario con las fechas por seriesId, para añadir al dataframe principal.

In [23]:
date = {
    '18406': '2021-10-12',
    '18407': '2021-10-12',
    '18670': '2021-10-12',
    '18408': '2021-10-27',
    '18409': '2021-10-27',
    '19335': '2021-10-28',
    '18673': '2021-10-28',
    '19336': '2021-10-28',
    '19374': '2021-10-29',
    '19375': '2021-10-29',
    '18674': '2021-10-29',
    '19404': '2021-10-30',
    '19408': '2021-10-30',
    '19407': '2021-10-31',
    '16259': '2021-08-11',
    '16258': '2021-08-11',
    '16261': '2021-08-11',
    '16260': '2021-08-11',
    '16580': '2021-08-11',
    '16583': '2021-08-11',
    '16582': '2021-08-12',
    '16581': '2021-08-12',
    '16602': '2021-08-12',
    '16601': '2021-08-12',
    '16584': '2021-08-13',
    '16609': '2021-08-13',
    '16637': '2021-08-14',
    '16638': '2021-08-14',
    '15828': '2021-07-29',
    '15829': '2021-07-29',
    '15831': '2021-07-29',
    '15830': '2021-07-29',
    '16044': '2021-07-29',
    '16045': '2021-07-29',
    '16047': '2021-07-30',
    '16046': '2021-07-30',
    '16048': '2021-07-30',
    '16049': '2021-07-30',
    '16053': '2021-07-31',
    '16056': '2021-07-31',
    '16110': '2021-07-31',
    '16111': '2021-08-01',
    '15569': '2021-07-22',
    '15584': '2021-07-22',
    '15575': '2021-07-22',
    '15583': '2021-07-22',
    '15571': '2021-07-22',
    '15572': '2021-07-22',
    '15573': '2021-07-22',
    '15574': '2021-07-22',
    '15576': '2021-07-22',
    '15578': '2021-07-22',
    '15580': '2021-07-22',
    '15582': '2021-07-22',
    '15585': '2021-07-22',
    '15586': '2021-07-22',
    '15591': '2021-07-22',
    '15587': '2021-07-22',
    '15592': '2021-07-22',
    '15593': '2021-07-22',
    '15594': '2021-07-22',
    '15595': '2021-07-22',
    '15596': '2021-07-22',
    '15597': '2021-07-22',
    '15645': '2021-07-22',
    '15599': '2021-07-22',
    '15600': '2021-07-22',
    '15601': '2021-07-22',
    '15604': '2021-07-22',
    '15607': '2021-07-22',
    '15610': '2021-07-22',
    '15613': '2021-07-22',
    '15619': '2021-07-22',
    '15647': '2021-07-22',
    '15639': '2021-07-22',
    '15649': '2021-07-22',
    '15642': '2021-07-22',
    '15650': '2021-07-22',
    '15651': '2021-07-22',
    '15652': '2021-07-22',
    '15655': '2021-07-22',
    '15661': '2021-07-22',
    '15658': '2021-07-22',
    '15659': '2021-07-22',
    '15621': '2021-07-22',
    '15614': '2021-07-22',
    '15616': '2021-07-22',
    '15618': '2021-07-22',
    '15620': '2021-07-22',
    '15622': '2021-07-22',
    '15627': '2021-07-22',
    '15623': '2021-07-22',
    '15625': '2021-07-22',
    '15626': '2021-07-22',
    '15628': '2021-07-22',
    '15629': '2021-07-22',
    '15630': '2021-07-22',
    '15631': '2021-07-22',
    '15570': '2021-07-22',
    '15577': '2021-07-22',
    '15617': '2021-07-22',
    '15624': '2021-07-22',
    '15632': '2021-07-22',
    '15790': '2021-07-22',
    '15751': '2021-07-22',
    '15758': '2021-07-22',
    '15745': '2021-07-22',
    '15746': '2021-07-22',
    '15683': '2021-07-22',
    '15685': '2021-07-22',
    '15688': '2021-07-22',
    '15689': '2021-07-22',
    '15714': '2021-07-22',
    '15670': '2021-07-22',
    '15663': '2021-07-22',
    '15780': '2021-07-22',
    '15781': '2021-07-22',
    '15791': '2021-07-22',
    '15673': '2021-07-22',
    '15678': '2021-07-22',
    '15754': '2021-07-22',
    '15759': '2021-07-22',
    '15662': '2021-07-22',
    '15761': '2021-07-22',
    '15605': '2021-07-22',
    '15608': '2021-07-22',
    '15611': '2021-07-22',
    '15612': '2021-07-22',
    '15778': '2021-07-22',
    '15637': '2021-07-22',
    '15606': '2021-07-22',
    '15615': '2021-07-22',
    '15609': '2021-07-22',
    '15638': '2021-07-22',
    '15779': '2021-07-22',
    '15774': '2021-07-23',
    '15775': '2021-07-23',
    '15776': '2021-07-23',
    '15772': '2021-07-23',
    '15777': '2021-07-23',
    '15769': '2021-07-23',
    '15770': '2021-07-23',
    '15765': '2021-07-23',
    '15762': '2021-07-23',
    '15764': '2021-07-23',
    '15773': '2021-07-23',
    '15763': '2021-07-23',
    '15768': '2021-07-23',
    '15766': '2021-07-23',
    '15767': '2021-07-23',
    '15771': '2021-07-23',
    '15783': '2021-07-23',
    '15788': '2021-07-23',
    '15785': '2021-07-23',
    '15787': '2021-07-23',
    '15784': '2021-07-23',
    '15782': '2021-07-23',
    '15789': '2021-07-23',
    '15786': '2021-07-23',
    '15795': '2021-07-24',
    '15798': '2021-07-24',
    '15796': '2021-07-25',
    '15797': '2021-07-25',
    '14520': '2021-07-08',
    '14519': '2021-07-08',
    '14521': '2021-07-08',
    '14522': '2021-07-08',
    '14766': '2021-07-08',
    '14767': '2021-07-08',
    '14770': '2021-07-09',
    '14769': '2021-07-09',
    '14800': '2021-07-09',
    '14799': '2021-07-09',
    '14801': '2021-07-10',
    '14802': '2021-07-10',
    '14816': '2021-07-10',
    '14817': '2021-07-11',
    '14167': '2021-07-01',
    '14117': '2021-07-01',
    '14116': '2021-07-01',
    '14139': '2021-07-01',
    '14140': '2021-07-01',
    '14141': '2021-07-01',
    '14142': '2021-07-01',
    '14157': '2021-07-01',
    '14143': '2021-07-01',
    '14144': '2021-07-01',
    '14145': '2021-07-01',
    '14146': '2021-07-01',
    '14148': '2021-07-01',
    '13907': '2021-07-01',
    '14174': '2021-07-01',
    '14176': '2021-07-01',
    '14177': '2021-07-01',
    '14178': '2021-07-01',
    '14131': '2021-07-01',
    '14180': '2021-07-01',
    '14118': '2021-07-01',
    '14153': '2021-07-01',
    '14154': '2021-07-01',
    '14125': '2021-07-01',
    '14128': '2021-07-01',
    '14155': '2021-07-01',
    '14129': '2021-07-01',
    '14175': '2021-07-01',
    '14120': '2021-07-01',
    '14156': '2021-07-01',
    '14135': '2021-07-01',
    '14138': '2021-07-01',
    '14130': '2021-07-01',
    '14171': '2021-07-01',
    '14119': '2021-07-01',
    '14158': '2021-07-01',
    '14124': '2021-07-01',
    '14126': '2021-07-01',
    '14149': '2021-07-01',
    '14136': '2021-07-01',
    '14164': '2021-07-01',
    '14159': '2021-07-01',
    '14121': '2021-07-01',
    '14160': '2021-07-01',
    '14168': '2021-07-01',
    '14172': '2021-07-01',
    '14165': '2021-07-01',
    '14161': '2021-07-01',
    '14152': '2021-07-01',
    '14170': '2021-07-01',
    '14163': '2021-07-01',
    '14134': '2021-07-01',
    '14132': '2021-07-01',
    '14151': '2021-07-01',
    '14166': '2021-07-01',
    '14127': '2021-07-01',
    '14169': '2021-07-01',
    '14162': '2021-07-01',
    '14182': '2021-07-01',
    '14246': '2021-07-01',
    '14181': '2021-07-01',
    '14150': '2021-07-01',
    '14147': '2021-07-01',
    '14245': '2021-07-01',
    '14229': '2021-07-01',
    '14234': '2021-07-01',
    '14247': '2021-07-01',
    '14235': '2021-07-01',
    '14236': '2021-07-01',
    '14183': '2021-07-01',
    '14133': '2021-07-01',
    '14249': '2021-07-01',
    '14189': '2021-07-01',
    '14191': '2021-07-01',
    '14187': '2021-07-01',
    '14195': '2021-07-01',
    '14192': '2021-07-01',
    '14193': '2021-07-01',
    '14196': '2021-07-01',
    '14184': '2021-07-01',
    '14190': '2021-07-01',
    '14185': '2021-07-01',
    '14194': '2021-07-01',
    '14238': '2021-07-01',
    '14188': '2021-07-01',
    '14248': '2021-07-01',
    '14250': '2021-07-01',
    '14186': '2021-07-01',
    '14224': '2021-07-02',
    '14217': '2021-07-02',
    '14216': '2021-07-02',
    '14222': '2021-07-02',
    '14214': '2021-07-02',
    '14226': '2021-07-02',
    '14227': '2021-07-02',
    '14228': '2021-07-02',
    '14219': '2021-07-02',
    '14218': '2021-07-02',
    '14220': '2021-07-02',
    '14223': '2021-07-02',
    '14215': '2021-07-02',
    '14221': '2021-07-02',
    '14213': '2021-07-02',
    '14225': '2021-07-02',
    '14271': '2021-07-03',
    '14270': '2021-07-03',
    '14272': '2021-07-03',
    '14273': '2021-07-03',
    '14275': '2021-07-04',
    '14276': '2021-07-04',
    '14277': '2021-07-04',
    '14278': '2021-07-04',
}    

In [24]:
df_valorant['seriesId'] = df_valorant['seriesId'].astype(str)

In [25]:
df_valorant['dates']= df_valorant['seriesId'].map(date)

In [26]:
df_valorant['seriesId'] = df_valorant['seriesId'].astype(int)

In [27]:
df_valorant['dates'] = pd.to_datetime(df_valorant['dates'])

In [28]:
df_valorant.head()

,eventId,seriesId,matchId,teamId,teamName,map,oppTeamId,oppTeamName,teamScore,atkFirst,...,clutchOpponents,clutchOpportunities,KDR,KD,FKD,KDA,spentCreds,loadoutValue,avgEconRating,dates
0,566,14278,29398,1137,VIRT,Ascent,3975,KCP,3,1,...,47,13,0.594203,-28,-4,41.202899,2536.875000,2989.375000,35.245134,2021-07-04
1,566,14278,29398,3975,KCP,Ascent,1137,VIRT,13,0,...,12,5,1.682927,28,4,69.658537,2355.000000,3820.625000,61.820594,2021-07-04
2,566,14278,29399,1137,VIRT,Bind,3975,KCP,5,0,...,54,15,0.537500,-37,-10,43.150000,2452.777778,2871.666667,37.879955,2021-07-04
3,566,14278,29399,3975,KCP,Bind,1137,VIRT,13,1,...,14,5,1.860465,37,10,80.418605,2179.444444,3838.333333,61.366301,2021-07-04
4,566,14277,29395,141,C9,Split,281,RISE,7,1,...,45,14,0.752941,-21,-4,64.341176,2618.500000,3167.500000,43.681497,2021-07-04


In [29]:
df_valorant = df_valorant.drop_duplicates()

In [30]:
duplicate = df_valorant[df_valorant.duplicated()]
 
print("Duplicate Rows :")
 
duplicate

Duplicate Rows :


,eventId,seriesId,matchId,teamId,teamName,map,oppTeamId,oppTeamName,teamScore,atkFirst,...,clutchOpponents,clutchOpportunities,KDR,KD,FKD,KDA,spentCreds,loadoutValue,avgEconRating,dates


In [31]:
df_valorant = df_valorant.drop(index=354)
df_valorant.reset_index(inplace=True)

In [32]:
df_valorant.sort_values(by='dates', inplace=True)

In [33]:
df_valorant.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1302 entries, 476 to 1228
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                1302 non-null   int64         
 1   eventId              1302 non-null   int64         
 2   seriesId             1302 non-null   int32         
 3   matchId              1302 non-null   int64         
 4   teamId               1302 non-null   int64         
 5   teamName             1302 non-null   object        
 6   map                  1302 non-null   object        
 7   oppTeamId            1302 non-null   int64         
 8   oppTeamName          1302 non-null   object        
 9   teamScore            1302 non-null   int64         
 10  atkFirst             1302 non-null   int64         
 11  won                  1302 non-null   int64         
 12  acs                  1302 non-null   float64       
 13  kills                1302 non-n

In [34]:
df_valorant.to_csv('df_valorant.csv')